## Exercise 7 - Final

This final exercise combines different parts from the previous exercises.

### Section 1

1 Search Digitraffic's REST interface service (in JSON format) for active trains with the terminal address `/live-trains` from Helsinki station with the following search parameters:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [80]:
import requests

station = 'HKI' # Helsinki 

arrived_trains = 10
departed_trains = 10
include_nonstopping = False

params = {
    'arrived_trains': arrived_trains,
    'departed_trains': departed_trains,
    'include_nonstopping': include_nonstopping
}

url = f"https://rata.digitraffic.fi/api/v1/live-trains/station/{station}/"

# https://rata.digitraffic.fi/api/v1/live-trains/station/HKI?version=0&arrived_trains=10&arriving_trains=5&departed_trains=10&departing_trains=5&include_nonstopping=false
req = requests.get(url, params)
req.status_code

200

2 Read the data into the Pandas dataframe without editing, so that the column headings are as follows:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version`

In [81]:
import pandas as pd
df = pd.json_normalize(req.json())

df.head()

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,6,2024-02-19,10,vr,IC,Long-distance,,False,False,287675616958,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'JNS', 'stationUICCode':..."
1,22,2024-02-19,10,vr,IC,Long-distance,,True,False,287675613459,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'ROI', 'stationUICCode':..."
2,954,2024-02-19,10,vr,S,Long-distance,,False,False,287675498714,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'KUT', 'stationUICCode':..."
3,955,2024-02-19,10,vr,IC,Long-distance,,True,False,287675605426,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
4,8106,2024-02-19,10,vr,HL,Commuter,A,False,False,287675614347,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'LPV', 'stationUICCode':..."


3 Delete all other columns except `timeTableRows` and `trainNumber`.

In [82]:
df = df.loc[:, ['timeTableRows', 'trainNumber']]
df.head()

,timeTableRows,trainNumber
0,"[{'stationShortCode': 'JNS', 'stationUICCode':...",6
1,"[{'stationShortCode': 'ROI', 'stationUICCode':...",22
2,"[{'stationShortCode': 'KUT', 'stationUICCode':...",954
3,"[{'stationShortCode': 'HKI', 'stationUICCode':...",955
4,"[{'stationShortCode': 'LPV', 'stationUICCode':...",8106


4 After this, extract the values ​​of the `timeTableRows` column from each train separately (the column value for each train contains several JSON objects) and add `trainNumber`, which identifies the train data.

In [85]:

# Extract data from nested JSON
new_data = []

for index, row in df.iterrows(): # loop over every row
    train_number = row['trainNumber']

    # loop over entries in timeTableRows[]
    for entry in row['timeTableRows']: # entry => one data row
        new_data_row = {}   
        new_data_row['trainNumber'] = row['trainNumber']
        for key, value in entry.items():
            new_data_row[key] = value
    
        new_data.append(new_data_row)

# Create a new DataFrame based on the extracted rows
# new_dataframe = pd.json_normalize(new_data)
new_dataframe = pd.DataFrame(new_data)
new_dataframe.head()


,trainNumber,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,actualTime,differenceInMinutes,causes,trainReady,liveEstimateTime,estimateSource
0,6,JNS,460,FI,DEPARTURE,True,True,1,False,2024-02-19T07:00:00.000Z,2024-02-19T07:00:00.000Z,0.0,[],"{'source': 'KUPLA', 'accepted': True, 'timesta...",NaN,NaN
1,6,PLT,1070,FI,ARRIVAL,False,NaN,,False,2024-02-19T07:00:42.000Z,NaN,NaN,[],NaN,NaN,NaN
2,6,PLT,1070,FI,DEPARTURE,False,NaN,,False,2024-02-19T07:00:42.000Z,NaN,NaN,[],NaN,NaN,NaN
3,6,SUL,1071,FI,ARRIVAL,False,NaN,,False,2024-02-19T07:01:32.000Z,NaN,NaN,[],NaN,NaN,NaN
4,6,SUL,1071,FI,DEPARTURE,False,NaN,,False,2024-02-19T07:01:32.000Z,NaN,NaN,[],NaN,NaN,NaN


5 Then drop the following columns from the data: `causes`, `differenceInMinutes` and `commercialStop`.

In [84]:
new_dataframe.drop(['causes', 'differenceInMinutes', 'commercialStop'], axis=1, inplace=True)

new_dataframe.head()

# df.head()

AttributeError: 'NoneType' object has no attribute 'head'

6 Fill in the missing values ​​of the `actualTime` column in each row with the values of the `scheduledTime` column.

In [ ]:
#

### Section 2

7 Extract the cell values ​​of the `table` element of the `data.html` file using the regex syntax and connect them to the dataframe created in step 1. **Note**: only regex should be used in the task (E.g. the use of Pandas' read_html() method is prohibited)!

In [ ]:
import re
tr_pattern = r"<tr[^>]*>(.*?)<\/tr>" # table rows, ignore attributes
th_pattern = r"<th>(.*?)<\/th>" # dataframe columns
td_pattern = r"<td>(.*?)<\/td>" # dataframe rows

with open("./data.html") as file:
    html_data = file.read()

columns = re.findall(th_pattern, html_data)

# rows = 
# table_data = []

# table_df = pd.DataFrame

# columns
html_data

'<!DOCTYPE html>\n\n<html>\n<head>\n    <meta charset="utf-8">\n    <style>\n        \n        table, th, td {\n            border: 1px solid black;\n            border-collapse: collapse;\n            }\n\n        table{\n            width: 75%;\n            margin-left: auto;\n            margin-right: auto;\n            text-align: center;\n        }\n\n        td{\n            padding: 5px;\n        }\n\n        #actualTime{\n            background-color: green\n        }\n    </style>\n\n</head>\n\n\n<header>\n\n</header>\n\n\n<body>\n\n    <table>\n        <tr id="actualTime">\n\n                <th>actualTime</th>\n                <th>cancelled</th>\n                <th>commercialTrack</th>\n                <th>countryCode</th>\n                <th>scheduledTime</th>\n                <th>stationShortCode</th>\n                <th>stationUICCode</th>\n                <th>trainStopping</th>\n                <th>type</th>\n                <th>trainNumber</th>\t\t\t\t\t\t\t\t\t\n   

### Section 3

8 Make a SQLite database and insert the dataframe obtained as a result of step 2 to the database.

In [ ]:
import sqlite3

# Create a database connection
db_connection = sqlite3.connect('final.db')

# Save dataframe to database
df.to_sql(name='live_trains', con=db_connection, if_exists='fail', index=False)

# Close the connect
db_connection.close()

AttributeError: 'NoneType' object has no attribute 'to_sql'